# Sony ZV1 camera assistant

### The Project

Creating a Chatbot that will assist you to get a grasp on a sony camera ZV1. In a way that it will answer your questions like "how to shoot in slow motion?", "How to set the black and white balance" ... so on so forth by reading the manual

The chatbot should :
* Accepts natural language questions about the Sony ZV-1.
* Returns relevant answers based solely on the official manual.
* Works offline or via a simple web app (optional).
* Is open-source and uses only free tech.

### 🧠 Project Workflow: Build a Manual-Based Q&A Assistant

 🗂️ 1. Extract Manual Content
- Download the PDF manual.
- Use **PyMuPDF** to extract clean text by section.
- Optionally add structure: title, section headers, etc.

🧩 2. Split & Embed
- Chunk the manual into coherent paragraphs using **LangChain’s `RecursiveCharacterTextSplitter`**.
- Generate embeddings with **SentenceTransformers**.

📚 3. Create Vector Store
- Store the chunks and their embeddings using **FAISS**.

🧠 4. Ask Questions
- User types a question.
- Perform **semantic search** on FAISS to retrieve relevant chunks.
- Feed these chunks + the question into an open-source LLM using a custom prompt:  
  *"Based on the following manual section, answer this question: [...]"*

### Technologies used (Free & Open-Source Stack)

| Purpose                     | Tool/Tech                                                                 |
|----------------------------|---------------------------------------------------------------------------|
| Text extraction from PDF   | PyMuPDF or pdfminer.six                                                  |
| Chunking + Embedding       | LangChain + SentenceTransformers (e.g. all-MiniLM-L6-v2)                 |
| Vector DB (search)         | FAISS (lightweight and local)                                            |
| LLM (to generate answers)  | llama.cpp or mistral-7b via Ollama (local) or OpenRouter (API access)    |

In [1]:
! jupyter kernelspec list

Available kernels:
  zv1-bot    /Users/guillaumegavillet/Library/Jupyter/kernels/zv1-bot
  python3    /Users/guillaumegavillet/.pyenv/versions/3.10.6/envs/lewagon/share/jupyter/kernels/python3


In [2]:
!jupyter kernelspec uninstall zmv1

Couldn't find kernel spec(s): zmv1


In [3]:
!y


zsh:1: command not found: y


## Extracting the text from the pdf manual

### Saving the text of the manual to a txt file

In [4]:
import fitz  # PyMuPDF
import os
import json
import re
import faiss
import numpy as np
import requests
from sentence_transformers import SentenceTransformer
from tqdm import tqdm

In [5]:
!which python

/Users/guillaumegavillet/.pyenv/versions/lewagon/bin/python


In [6]:


def extract_text_from_pdf(pdf_path, output_txt_path=None):
    """
    Extracts text from a PDF file using PyMuPDF.
    
    Args:
        pdf_path (str): Path to the PDF file.
        output_txt_path (str): Optional path to save the extracted text as a .txt file.
    
    Returns:
        str: Full extracted text.
    """
    doc = fitz.open(pdf_path)
    full_text = ""

    for page_num, page in enumerate(doc):
        text = page.get_text()
        full_text += f"\n--- Page {page_num + 1} ---\n{text}"

    if output_txt_path:
        with open(output_txt_path, 'w', encoding='utf-8') as f:
            f.write(full_text)
        print(f"✅ Text saved to: {output_txt_path}")
    
    return full_text

# Example usage:
if __name__ == "__main__":
    pdf_path = "data/zv1_manual.pdf"  # file path
    extract_text_from_pdf(pdf_path, output_txt_path="sony_zv1_text.txt")


✅ Text saved to: sony_zv1_text.txt


checking the first 1000 characters of the text, to see if it went well

In [7]:
with open("sony_zv1_text.txt", "r", encoding="utf-8") as f:
    content = f.read()
    print(content[15000:20000])



y dealer or local authorized Sony service facility.
14.
HDMI micro jack
15.
Wi-Fi/Bluetooth antenna (built-in)
16.
For shooting: Fn (Function) button  
For viewing: 
 (Send to Smartphone) button
17.
MENU button
18.
20

--- Page 21 ---
Using the wind screen (supplied)
Use the wind screen to reduce wind noise picked up by the internal microphone when recording a movie.
Remove the shoe cap from the camera, and then attach the wind screen to the Multi Interface Shoe.
Note
When attaching the wind screen, make sure that the wind screen does not get caught in the Multi Interface Shoe.
Be careful not to lose the shoe cap when you remove it from the camera.
5-016-934-12(1) Copyright 2020 Sony Corporation
Control wheel
19.
Center button
20.
 (Playback) button
21.
C2/
 (Custom button 2/Delete) button
22.
Hook for strap
23.
Battery insertion slot
24.
Battery lock lever
25.
Access lamp
26.
Memory card slot
27.
Tripod socket hole 
Supports 1/4-20 UNC screws
Use a tripod with a screw less than 5.5 mm

## Chunking the manual for segmental search 

### Why chunking
we chunk the text (divide it in smaller section) because llm can't handle everything at once Most have context length limits, and even outside of that, searching smaller pieces of information gives better results.

In [8]:
def split_text_into_chunks(text, chunk_size=700, overlap=150):
    """
    Splits the input text into overlapping chunks.
    """
    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        chunk = text[start:end]
        chunks.append(chunk.strip())
        start += chunk_size - overlap
    return chunks


In [9]:
def extract_chunks_from_txt_file(txt_path, output_path):
    """
    Extracts page-wise text from a TXT file (with '--- Page X ---' markers),
    splits the content into chunks, and saves to a JSONL file.
    """
    with open(txt_path, 'r', encoding='utf-8') as f:
        raw_text = f.read()

    # Split by pages using the "--- Page X ---" pattern
    pages = re.split(r'--- Page \d+ ---', raw_text)

    all_chunks = []
    for i, page_text in enumerate(pages):
        page_text = page_text.strip()
        if not page_text:
            continue

        chunks = split_text_into_chunks(page_text)
        for j, chunk in enumerate(chunks):
            all_chunks.append({
                "page": i + 1,
                "chunk_id": f"page_{i+1}_chunk_{j+1}",
                "text": chunk
            })

    # Save to JSON Lines file
    with open(output_path, 'w', encoding='utf-8') as out_file:
        for chunk in all_chunks:
            json.dump(chunk, out_file)
            out_file.write('\n')

    print(f"✅ Saved {len(all_chunks)} chunks to: {output_path}")

In [10]:
extract_chunks_from_txt_file("sony_zv1_text.txt", "zv1_chunks.jsonl")

✅ Saved 1346 chunks to: zv1_chunks.jsonl


## Embed the chunks and save them into a FAISS index

* Load your zv1_chunks.jsonl

* Embed each chunk using SentenceTransformer

* Save the FAISS index to zv1_faiss.index
    * the Faiss is the library in charge of identifying the similarities between the query an

* Save the metadata (chunk text + page info) to zv1_metadata.json

In [11]:
def load_chunks(jsonl_path):
    chunks = []
    with open(jsonl_path, 'r', encoding='utf-8') as f:
        for line in f:
            chunks.append(json.loads(line.strip()))
    return chunks

def embed_chunks(chunks, model_name="all-MiniLM-L6-v2"):
    model = SentenceTransformer(model_name)
    texts = [chunk['text'] for chunk in chunks]
    embeddings = model.encode(texts, show_progress_bar=True)
    return np.array(embeddings).astype('float32')

def save_faiss_index(embeddings, index_path="zv1_faiss.index"):
    dim = embeddings.shape[1]
    index = faiss.IndexFlatL2(dim)
    index.add(embeddings)
    faiss.write_index(index, index_path)
    print(f"✅ FAISS index saved to: {index_path}")
    return index

def save_metadata(chunks, path="zv1_metadata.json"):
    with open(path, 'w', encoding='utf-8') as f:
        json.dump(chunks, f, indent=2)
    print(f"✅ Metadata saved to: {path}")

In [12]:
chunks = load_chunks("zv1_chunks.jsonl")
embeddings = embed_chunks(chunks)
save_faiss_index(embeddings, "zv1_faiss.index")
save_metadata(chunks, "zv1_metadata.json")

Batches:   0%|          | 0/43 [00:00<?, ?it/s]

✅ FAISS index saved to: zv1_faiss.index
✅ Metadata saved to: zv1_metadata.json


In [13]:
def load_index(index_path):
    return faiss.read_index(index_path)

def load_metadata(path):
    with open(path, 'r', encoding='utf-8') as f:
        return json.load(f)

def search(query, index, metadata, model, top_k=3):
    query_embedding = model.encode([query]).astype('float32')
    distances, indices = index.search(query_embedding, top_k)

    results = []
    for i in range(top_k):
        idx = indices[0][i]
        results.append({
            "score": float(distances[0][i]),
            "page": metadata[idx]["page"],
            "chunk_id": metadata[idx]["chunk_id"],
            "text": metadata[idx]["text"]
        })
    return results

## Model

the model here is used to embed the query : Convert your question into a vector → for searching similar chunks in FAISS.

In [14]:

model = SentenceTransformer("all-MiniLM-L6-v2")
index = load_index("zv1_faiss.index")
metadata = load_metadata("zv1_metadata.json")

## Queries and fun

Here I am asking some questions to the model and trying to see how it performs 

The model for embedding the query: 
* "all-MiniLM-L6-v2" https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2
* from Hugging Face




1. We're embedding the query using ""all-MiniLM-L6-v2"
2. Searching FAISS with that vector :  searching in the manual the similarity between query and the embedded manual. This is a library using a knn algorithm and returning L2 between nearest neighbour and query 
3. Printing the chunks : nearest neighbours, of the query plus the L2 scores 

In [15]:
def results_query(query, index, metadata, model):
    results = search(query, index, metadata, model)
    scores = []
    texts = []
    for res in results:
        print(f"\n🔎 Page {res['page']} | Score: {res['score']:.2f}")
        print(res["text"])
        scores.append(res['score'])
        texts.append(res["text"])
    return None
    

In [16]:
query = "How do I enable slow motion video?"
results = search(query, index, metadata, model)

for res in results:
    print(f"\n🔎 Page {res['page']} | Score: {res['score']:.2f}")
    print(res["text"])


🔎 Page 232 | Score: 0.76
Help Guide
Digital Camera
ZV-1
Motion Shot Video
You can see the tracking of the subject's motion in high-speed, like a stroboscopic image.
Hint
You can also change the interval of the image tracking using MENU → 
 (Playback) → [Motion Interval ADJ].
Note
You cannot save the images made with [Motion Shot Video] as a movie file.
If the motion of the subject is too slow or the subject does not move enough, the product may fail to create the image.
Related Topic
Motion Interval ADJ
5-016-934-12(1) Copyright 2020 Sony Corporation
Press the bottom side of the control wheel during movie playback, then select 
 (Motion Shot Video).
To exit [Motion Shot Video] playback, select 
.
If you fail to make

🔎 Page 189 | Score: 0.76
Help Guide
Digital Camera
ZV-1
Shooting super-slow-motion movies (HFR Settings)
By shooting with a higher frame rate than the recording format, you can record a smooth super-slow-motion movie.
MODE (Mode) button → [High Frame Rate] → press the cen

In [17]:
query = "how to modify black and white balance ?"
results = search(query, index, metadata, model)

for res in results:
    print(f"\n🔎 Page {res['page']} | Score: {res['score']:.2f}")
    print(res["text"])


🔎 Page 207 | Score: 0.75
you increase the setting value towards the positive side, and lighter as you
decrease the value towards the negative side. This function is effective even if you set [Color Mode] to [Black & White].
[R] -7 (light red) to +7 (deep red)
[G] -7 (light green) to +7 (deep green)
[B] -7 (light blue) to +7 (deep blue)
[C] -7 (light cyan) to +7 (deep cyan)
[M] -7 (light magenta) to +7 (deep magenta)
[Y] -7 (light yellow) to +7 (deep yellow)
Detail
Sets items for [Detail].
Level: Sets the [Detail] level. (-7 to +7)
Adjust: The following parameters can be selected manually.
Mode: Selects auto/manual setting. (Auto (automatic optimization) / Manual (The details are set manually.))
V/H Balance: Sets th

🔎 Page 394 | Score: 0.75
you increase the setting value towards the positive side, and lighter as you
decrease the value towards the negative side. This function is effective even if you set [Color Mode] to [Black & White].
[R] -7 (light red) to +7 (deep red)
[G] -7 (light

In [18]:
query = "how to modify black and white balance ?"
results_query(query, index, metadata, model)


🔎 Page 207 | Score: 0.75
you increase the setting value towards the positive side, and lighter as you
decrease the value towards the negative side. This function is effective even if you set [Color Mode] to [Black & White].
[R] -7 (light red) to +7 (deep red)
[G] -7 (light green) to +7 (deep green)
[B] -7 (light blue) to +7 (deep blue)
[C] -7 (light cyan) to +7 (deep cyan)
[M] -7 (light magenta) to +7 (deep magenta)
[Y] -7 (light yellow) to +7 (deep yellow)
Detail
Sets items for [Detail].
Level: Sets the [Detail] level. (-7 to +7)
Adjust: The following parameters can be selected manually.
Mode: Selects auto/manual setting. (Auto (automatic optimization) / Manual (The details are set manually.))
V/H Balance: Sets th

🔎 Page 394 | Score: 0.75
you increase the setting value towards the positive side, and lighter as you
decrease the value towards the negative side. This function is effective even if you set [Color Mode] to [Black & White].
[R] -7 (light red) to +7 (deep red)
[G] -7 (light

In [19]:
query = "how to avoid rolling shutter effect ?"
results_query(query, index, metadata, model)


🔎 Page 433 | Score: 0.93
Help Guide
Digital Camera
ZV-1
SteadyShot (movie)
Sets [
SteadyShot] effect when shooting movies. If you set the [
SteadyShot] effect to [Off] when using a tripod
(sold separately), a natural image is produced.
Menu item details
Active:
Provides a more powerful SteadyShot effect.
Standard:
Reduces camera shake under stable movie shooting conditions.
Off:
Does not use [
SteadyShot].
Note
If you change the setting for [
SteadyShot], the angle of view will change.
Related Topic
SteadyShot (still image)
5-016-934-12(1) Copyright 2020 Sony Corporation
MENU → 
 (Camera Settings2) → [
SteadyShot] → desired setting.
1
432

🔎 Page 214 | Score: 0.93
Help Guide
Digital Camera
ZV-1
SteadyShot (movie)
Sets [
SteadyShot] effect when shooting movies. If you set the [
SteadyShot] effect to [Off] when using a tripod
(sold separately), a natural image is produced.
Menu item details
Active:
Provides a more powerful SteadyShot effect.
Standard:
Reduces camera shake under stable m

In [20]:
query = "How do i shoot in low light conditions ? "
results_query(query, index, metadata, model)


🔎 Page 122 | Score: 1.10
Help Guide
Digital Camera
ZV-1
Bulb shooting
You can shoot a trailing image of the movement of a subject with long exposure.
Bulb shooting is suitable for shooting trails from stars or fireworks, etc.
Hint
When shooting fireworks, etc., focus at infinity in manual focus mode.
In order to perform bulb shooting without causing the image quality to deteriorate, we recommend that you start shooting while
the camera is cool.
During bulb shooting, images tend to blur. It is recommended that you use a tripod, a RMT-P1BT Bluetooth remote commander
(sold separately) or a remote commander equipped with a lock function (sold separately). When using the RMT-P1BT Bluetooth
remote commander, you can start

🔎 Page 127 | Score: 1.11
in sharp focus with vivid colors. 
Sunset:
Shoots the red of the sunset beautifully. 
MODE (Mode) button → [Scene Selection] → press the center of the control wheel.
You can also set the shooting mode by selecting MENU → 
 (Camera Settings1) → [Sh

## Add an LLM to sum up, explain the essence of the response

We then use the mistral model from Ollama to use the text retrieved from the query and turn it into a summarize to the point "explanation"


In [21]:
# give the context to the mistral ai using the prompt
# the prompt can be modified to suit the person using the camera (an expert or a kid for instance)

def build_prompt(question, retrieved_chunks):
    context = "\n\n".join([
        f"[Page {chunk['page']}] {chunk['text']}" for chunk in retrieved_chunks
    ])

    prompt = f"""You are a helpful assistant answering based on the Sony ZV-1 camera manual.

Use the following manual content to answer the user's question as clearly and accurately as possible. Add 
the section from the manual (like page) from which the information comes from in between brackets.  

Manual Content:
{context}


Question: {question}
Answer:"""
    


    return prompt

In [22]:
# do the api query, using the prompt and the model

def query_ollama(prompt, model="mistral"):
    url = "http://localhost:11434/api/generate"
    payload = {
        "model": model,
        "prompt": prompt,
        "stream": False
    }
    try:
        response = requests.post(url, json=payload)
        response.raise_for_status()
        return response.json()["response"]
    except Exception as e:
        return f"Error querying Ollama: {e}"

In [23]:
# main function that call the different functions and answer accordingly 

def ask(question, index, metadata, embedding_model, llm_model_name="mistral", top_k=3):
    
    # Step 1: Embed query
    query_embedding = embedding_model.encode([question]).astype("float32")

    # Step 2: Search FAISS
    distances, indices = index.search(query_embedding, top_k)
    retrieved_chunks = [metadata[idx] for idx in indices[0]]

    # Step 3: Build prompt
    prompt = build_prompt(question, retrieved_chunks)

    # Step 4: Get answer from LLM
    answer = query_ollama(prompt, model=llm_model_name)

    # Step 5: Print everything
    print("\n🔍 Retrieved chunks:")
    for i, chunk in enumerate(retrieved_chunks):
        print(f"\n[Page {chunk['page']}] (score: {distances[0][i]:.2f})")
        print(chunk['text'])

    print("\n🤖 LLM Answer:")
    print(answer)
    return answer

In [24]:
query = "how to have a blurred background ?"
ask(query, index, metadata, model)


🔍 Retrieved chunks:

[Page 119] (score: 0.96)
Help Guide
Digital Camera
ZV-1
Aperture Priority
You can shoot by adjusting the aperture and changing the range in focus, or by defocusing the background.
Note
The brightness of the image on the screen may differ from the actual image being shot.
5-016-934-12(1) Copyright 2020 Sony Corporation
MODE (Mode) button → [Aperture Priority] → press the center of the control wheel.
You can also set the shooting mode by selecting MENU → 
 (Camera Settings1) → [Shoot Mode].
1
Select the desired value by turning the control wheel.
Smaller F-value: The subject is in focus, but objects in front of and beyond the subject are blurred.  
Larger F-value: The subject and its foreground and background are all

[Page 82] (score: 0.99)
o be shown magnified by selecting MENU → 
 (Camera Settings1) →
[Focus Magnif. Time].
3
Press the shutter button fully down to shoot the image.
4
81

[Page 397] (score: 0.99)
o be shown magnified by selecting MENU → 
 (Camera Se

" To achieve a blurred background (defocused background), you should use Aperture Priority mode on your Sony ZV-1 camera. Here's a step-by-step guide:\n\n1. Press the MODE button and select the Aperture Priority mode by rotating the control wheel until it shows 'A'. (Refer to Page 119 of the user manual)\n2. Now, you can adjust the aperture value (F-value). A smaller F-value results in a shallow depth of field, making the background blurred while keeping your subject sharp and in focus. Conversely, a larger F-value will keep more of your image in focus, including the background.\n\nNote: The brightness of the image on the screen may differ from the actual image being shot. (Refer to Page 119 of the user manual)\n\nYou can also change the shooting mode by selecting MENU -> Camera Settings1 -> [Shoot Mode], and then rotating the control wheel to choose your desired value. (Refer to Page 119 of the user manual)\n\nFor more precise focus, you might want to adjust the Focus Magnification Ti

In [25]:
query = "how to have shoot a video in manual mode?"
ask(query, index, metadata, model)


🔍 Retrieved chunks:

[Page 133] (score: 0.77)
Copyright 2020 Sony Corporation
MODE (Mode) button → [Movie] → press the center of the control wheel.
You can also set the shooting mode by selecting MENU → 
 (Camera Settings1) → [Shoot Mode].
1
MENU→ 
 (Camera Settings2) → [
Exposure Mode] → desired setting.
2
Press the MOVIE (movie) button to start recording.
  
Press the MOVIE button again to stop recording.
3
132

[Page 410] (score: 0.78)
Copyright 2020 Sony Corporation
MODE (Mode) button → [Movie] → press the center of the control wheel.
You can also set the shooting mode by selecting MENU → 
 (Camera Settings1) → [Shoot Mode].
1
MENU→ 
 (Camera Settings2) → [
Exposure Mode] → desired setting.
2
Press the MOVIE (movie) button to start recording.
  
Press the MOVIE button again to stop recording.
3
409

[Page 134] (score: 0.80)
lligent Auto] → press the center of the control wheel.
You can also set the shooting mode by selecting MENU → 
 (Camera Settings1)→ [Shoot Mode].
1
Point the c

' To shoot a video in Manual Mode on your Sony ZV-1 camera, follow these steps:\n\n1. Press the MODE (Mode) button [Page 133 or Page 410].\n2. Navigate to [Movie] using the control wheel and press the center of the control wheel.\n3. Alternatively, you can set the shooting mode by selecting MENU:\n   a. Go to (Camera Settings1) and select [Shoot Mode].\n   b. Then, go to (Camera Settings2) and select [Exposure Mode].\n   c. Choose your desired Manual (M) setting.\n4. Press the MOVIE (movie) button to start recording.\n5. Press the MOVIE button again to stop recording.'

In [27]:
query = "how to shoot in slow motion ?"
ask(query, index, metadata, model)


🔍 Retrieved chunks:

[Page 190] (score: 0.90)
button ([Start Trigger]), or for a set amount
of time until you press the MOVIE button ([End Trigger]/[End Trigger Half]).
Frame rate
In super-slow-motion movie shooting, the camera shoots at a faster shutter speed than the number of shooting frames
per second. For example, when [
Frame Rate] is set to [960fps], the shutter speed per frame will be faster than
approx. 1/1000 second in order to shoot 960 frames per second. To maintain this shutter speed, sufficient ambient light is
necessary during shooting. If the ambient light is insufficient, the ISO sensitivity will become higher, resulting in more
noise.
Shortest shooting distance
The image becomes out of focus when the subject is too cl

[Page 418] (score: 0.90)
button ([Start Trigger]), or for a set amount
of time until you press the MOVIE button ([End Trigger]/[End Trigger Half]).
Frame rate
In super-slow-motion movie shooting, the camera shoots at a faster shutter speed than the num

" To shoot in slow motion, also known as super-slow-motion movie shooting, follow these steps from the Sony ZV-1 camera manual (Page 418 and Page 5):\n\n1. Press the MODE (Mode) button to access the camera modes.\n2. Navigate to [High Frame Rate] by pressing the control wheel's center.\n3. Set the desired frame rate for slow motion in the following steps. For instance, if you want to shoot at 960fps, follow these additional instructions:\n   a. In Camera Settings1 (MENU → ), select [Shoot Mode].\n   b. Set the shooting mode to your preferred mode (for example, P for Program Auto or A for Shutter-priority Auto).\n   c. In Camera Settings2 (MENU → ), select [Exposure Mode] and set it to a suitable value depending on the lighting conditions. Remember that sufficient ambient light is necessary during shooting to maintain the required shutter speed, as the camera shoots at a faster speed than the chosen frame rate.\n   d. If the ambient light is insufficient, the ISO sensitivity will increa

In [28]:
query = "how do i do a long exposure ?"
ask(query, index, metadata, model)


🔍 Retrieved chunks:

[Page 149] (score: 1.06)
n the contrast of the subject. Select the optimization level from 1.0EV (weak) to
6.0EV (strong).  
For example, if you set the exposure value to 2.0 EV, three images will be composed with the following exposure levels:
-1.0 EV, correct exposure and +1.0 EV.
Hint
The shutter is released three times for one shot. Be careful about the following:
Use this function when the subject is motionless or does not flash light.
Do not change the composition during shooting.
Note
This function is not available when the [
File Format] is [RAW] or [RAW & JPEG].
[Auto HDR] is not available in the following shooting modes:
[Intelligent Auto]
[Sweep Panorama]
[Scene Selection]
[Auto HDR] is not available in t

[Page 387] (score: 1.06)
n the contrast of the subject. Select the optimization level from 1.0EV (weak) to
6.0EV (strong).  
For example, if you set the exposure value to 2.0 EV, three images will be composed with the following exposure levels:
-1.0 E

" To perform a long exposure on your Sony ZV-1 camera, you will need to adjust the Shutter Speed settings manually since Auto mode usually doesn't allow for extended exposures. Here are the steps to achieve a long exposure:\n\n1. Turn on your camera and make sure it is set to Manual (M) mode [Page 81].\n2. Use the main dial located at the top of the camera to adjust the shutter speed. Set it to a slow speed, such as 1 second, 5 seconds, or longer. The slower the shutter speed, the longer the exposure. You can set the shutter speed up to 30 seconds in Manual mode [Page 89].\n3. Adjust the Aperture (Av) and ISO settings to achieve a correct exposure based on the available light. A tripod or stabilizer is highly recommended for longer exposures to prevent camera shake [Page 273-275].\n4. Press the shutter button fully down to take the image.\n\nPlease note that long exposures may not be possible during extremely bright conditions due to the camera's sensor capabilities, and you may need a